In [181]:
import numpy as np
import os

from PIL import Image

PATH = "../benchmarks_2/id_muc_scaling_big"
ASSUMPTION_SIGNATURE = "a"
CUSTOM_UNSAT_HEAD = ""

In [205]:
n_assumptions = 10000
n_mucs = 10

density = (0.5 / (n_assumptions / 100))
verbose = 0

mucs = []

while True:
    assignment = np.random.binomial(n=1, p=density, size=n_assumptions)
    candidate = {i + 1 for i, x in enumerate(assignment) if x == 1}
    if sum(assignment) != 50:
        continue

    if verbose:
        print(assignment, candidate)

    if any([candidate.issubset(muc) or muc.issubset(candidate) for muc in mucs]):
        if verbose:
            print("DENIED")
    else:
        if verbose:
            print("APPROVED")
        mucs.append(candidate)
        if len(mucs) >= n_mucs:
            break

In [206]:
print(len(mucs))
for muc in mucs:
    print(muc)

assumption_dir = f"{n_assumptions}_assumptions"
if PATH:
    assumption_dir = f"{PATH}/{assumption_dir}"
muc_dir = f"{assumption_dir}/{n_mucs}_mucs"
dirname = muc_dir

if not os.path.isdir(assumption_dir):
    os.mkdir(assumption_dir)
if not os.path.isdir(muc_dir):
    os.mkdir(muc_dir)


img_width = 1280
img_height = int((n_mucs / n_assumptions) * img_width)
img = np.zeros(shape=(n_mucs, n_assumptions), dtype=np.uint8) + 255
for m, muc in enumerate(mucs):
    for a in muc:
        img[m, a - 1] = 0
Image.fromarray(img).resize((img_width, img_height), resample=Image.BOX).save(f"{dirname}/visualization.jpeg")

with open(f"{dirname}/encoding.lp", "w") as f:
    f.write("% ---[ AUTO GENERATED MUC BENCHMARK ]\n")
    f.write(f"% ---[ assumptions : {n_assumptions} ]\n")
    f.write(f"% ---[ mucs : {n_mucs} ]\n\n")

    f.write(f"num(1..{n_assumptions}).\n")
    f.write("{" + ASSUMPTION_SIGNATURE + "(X): num(X)}.\n\n")

    for muc in mucs:
        f.write(f"{CUSTOM_UNSAT_HEAD}:- " + ", ".join([f"{ASSUMPTION_SIGNATURE}({a})" for a in muc]) + ".\n")

with open(f"{dirname}/assumptions.lp", "w") as f:
    for a in range(n_assumptions):
        f.write(f"assume({ASSUMPTION_SIGNATURE}({a + 1})).\n")

with open(f"{dirname}/results.txt", "w") as f:
    results = dict()
    results["minimal"] = [list([f"{ASSUMPTION_SIGNATURE}({a})" for a in muc]) for muc in mucs]

    minimum = [muc for muc in mucs if len(muc) == len(min(mucs, key=lambda x: len(x)))]
    results["minimum"] = [list([f"{ASSUMPTION_SIGNATURE}({a})" for a in muc]) for muc in minimum]

    f.write(str(results).replace("'", '"'))

with open(f"{assumption_dir}/README.md", "w") as f:
    f.write(f"# Random MUC Benchmarks [{n_assumptions} Assumptions]\n\n")
    for s, subdir in enumerate(os.walk(assumption_dir)):
        if subdir[0] != assumption_dir:
            subdir_mucs = int(subdir[0].split("/")[-1].replace("_mucs", ""))
            f.write(f"## {s} : {subdir_mucs} MUCs\n\n")
            f.write(f"![]({subdir[0].split('/')[-1]}/visualization.jpeg)\n\n")

10
{3077, 3078, 1160, 1528, 8456, 8584, 5516, 9096, 401, 5265, 789, 9851, 2590, 544, 3745, 8098, 6822, 7079, 6056, 4396, 5296, 820, 8251, 6077, 7613, 9540, 8006, 6985, 5455, 2384, 2640, 6737, 2131, 2644, 7129, 2526, 8440, 8418, 9699, 743, 4586, 9450, 5868, 7927, 4344, 4601, 6266, 6011, 2045, 126}
{7555, 4484, 3589, 4740, 2317, 7443, 9620, 9877, 2455, 5274, 3099, 8090, 2721, 4260, 809, 1713, 5041, 2484, 9013, 3766, 6458, 4284, 1090, 3138, 9928, 4809, 9803, 7373, 5454, 2127, 6863, 594, 6226, 7763, 6871, 3677, 3165, 733, 6885, 5350, 1127, 1767, 8681, 9573, 5743, 8047, 9713, 373, 9846, 1660}
{2689, 6273, 1027, 1293, 8333, 6672, 2068, 9112, 3360, 1697, 7586, 1958, 9895, 2996, 6709, 9268, 1596, 61, 1469, 1471, 6214, 6726, 330, 5323, 8012, 594, 2132, 6356, 3670, 4439, 4056, 4441, 6104, 6105, 92, 6741, 9688, 8287, 4576, 353, 4962, 7784, 1009, 9202, 3317, 3447, 7161, 6011, 9980, 6996}
{6407, 3721, 1930, 2955, 3981, 6030, 911, 7058, 787, 536, 5402, 415, 9637, 1451, 9388, 6831, 9010, 8886, 3895, 